In [1]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [15]:
import os
import json
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    Trainer,
    TrainingArguments,
    DataCollatorForSeq2Seq,
    EarlyStoppingCallback
)
from datasets import Dataset
import torch

### Define Paths and Model Name

In [3]:
DATA_PATH = "/content/fine_tune_dataset.json"
OUTPUT_DIR = "/content/fine_tuned_model"

### Load Data Function

In [4]:
def load_data(data_path):
    """Load the dataset from JSON."""
    with open(data_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    return Dataset.from_list(data)

### Tokenization Function

In [5]:
def tokenize_data(examples, tokenizer, max_input_length=512, max_output_length=128):
    """
    Tokenize the dataset.
    - Inputs: Subject + Text so far
    - Outputs: Next words
    """
    inputs = examples["input"]  # Extract the 'input' field from the batch
    outputs = examples["output"]  # Extract the 'output' field from the batch
    model_inputs = tokenizer(
        inputs,
        max_length=max_input_length,
        padding="max_length",
        truncation=True,
    )
    labels = tokenizer(
        outputs,
        max_length=max_output_length,
        padding="max_length",
        truncation=True,
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

### Load Dataset and Model

In [6]:
MODEL_NAME = 'facebook/bart-base'

In [7]:
# Load dataset
print(f"Loading dataset from {DATA_PATH}...")
dataset = load_data(DATA_PATH)
print(f"Loaded {len(dataset)} examples.")

# Load tokenizer and model
print(f"Loading model and tokenizer ({MODEL_NAME})...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

Loading dataset from /content/fine_tune_dataset.json...
Loaded 21674 examples.
Loading model and tokenizer (facebook/bart-base)...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

### Tokenize the Dataset

In [8]:
# Tokenize dataset
print("Tokenizing dataset...")
tokenized_dataset = dataset.map(
    lambda examples: tokenize_data(examples, tokenizer),
    batched=True,
    remove_columns=["input", "output"],
)

# Data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

Tokenizing dataset...


Map:   0%|          | 0/21674 [00:00<?, ? examples/s]

### Define Training Arguments

In [22]:
print("Setting up training arguments...")
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    evaluation_strategy="steps",  # Ensure evaluation happens during training
    eval_steps=2000,  # Adjust based on dataset size and training speed
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,  # Set a higher number; early stopping will stop earlier if needed
    save_total_limit=2,
    logging_steps=2000,
    save_steps=2000,
    save_strategy="steps",
    fp16=True if torch.cuda.is_available() else False,
    load_best_model_at_end=True,  # Enable loading the best model
    metric_for_best_model="eval_loss",  # Metric to monitor (change as needed)
    greater_is_better=False,  # Set to False for loss, True for accuracy, etc.
    report_to="wandb",  # If using Weights and Biases
)

Setting up training arguments...


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


### Initialize and Run Trainer

In [23]:
print("Initializing Trainer...")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],  # Stop after 3 non-improving evals
)

Initializing Trainer...


In [ ]:
# Train
print("Starting fine-tuning...")
trainer.train()

Starting fine-tuning...


Step,Training Loss,Validation Loss
500,0.066500,0.162446
1000,0.114200,0.135349
1500,0.176500,0.122891
2000,0.163700,0.107169
2500,0.153600,0.095309
3000,0.128700,0.083425
3500,0.113000,0.075364
4000,0.109800,0.067262
4500,0.102800,0.061101
5000,0.098300,0.054704


Step,Training Loss,Validation Loss
500,0.066500,0.162446
1000,0.114200,0.135349
1500,0.176500,0.122891
2000,0.163700,0.107169
2500,0.153600,0.095309
3000,0.128700,0.083425
3500,0.113000,0.075364
4000,0.109800,0.067262
4500,0.102800,0.061101
5000,0.098300,0.054704


### Save the Fine-Tuned Model

In [ ]:
print(f"Saving model to {OUTPUT_DIR}...")
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print("Model saved.")